# Download and import Libraries

In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

In [3]:
import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "johnjehiel/personal-health-LLM-Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = HF_TOKEN
)

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Unsloth 2025.3.14 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [6]:
# Template for prompting
prompt_template = """Below is an instruction that describes a task, paired with an optional input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Testing

## 1B Model

In [7]:
from datasets import load_dataset

In [8]:
ds = load_dataset("johnjehiel/ph-llm-dataset-batch-splits")["test"]

train_batch_1.parquet:   0%|          | 0.00/8.34M [00:00<?, ?B/s]

train_batch_2.parquet:   0%|          | 0.00/8.39M [00:00<?, ?B/s]

train_batch_3.parquet:   0%|          | 0.00/8.37M [00:00<?, ?B/s]

train_batch_4.parquet:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

train_batch_5.parquet:   0%|          | 0.00/8.34M [00:00<?, ?B/s]

...

test_batch_3.parquet:   0%|          | 0.00/754k [00:00<?, ?B/s]

test_batch_4.parquet:   0%|          | 0.00/768k [00:00<?, ?B/s]

test_batch_5.parquet:   0%|          | 0.00/755k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67301 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5934 [00:00<?, ? examples/s]

In [35]:
d = {}
for i, category in enumerate(ds["Category"]):
    if category not in d:
        d[category] = i
d

{'A1': 0,
 'A2': 9,
 'A3': 18,
 'A6': 27,
 'MedMCQA': 35,
 'anatomy': 135,
 'college_biology': 138,
 'college_medicine': 142,
 'medical_genetics': 147,
 'medqa': 150,
 'mmlu_clinical_knowledge': 405,
 'patient_doctor_conversation': 411,
 'professional_medicine': 1175,
 'pubmedqa': 1182}

In [36]:
ds["Prompt"][27]

'You are a NSCA and ACSM board-certified fitness trainer who specializes in athlete training performance and recovery. \nAge: 43  \nGender: Male\nSleep Duration: 6.5\nQuality of Sleep: 6\nPhysical Activity Level: 45\nStress Level: 7\nBMI Category: Overweight \nBlood Pressure: 130/85\nHeart Rate: 72\nDaily Steps: 6000\n\nAre there any special precautions that should be taken into account when recommending a fitness program to avoid injury? Comment if the trainee has exceptional demographics (e.g. very old, very high BMI, very low BMI) that require special considerations. Write a single sentence. Avoid mentioning diseases\n'

In [25]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
def generate_response(prompt_input, expected_response):
    inputs = tokenizer(
    [
        prompt_template.format(
            [prompt_input],
            "", # input
            "", # output - leave this blank for generation
        )
    ], return_tensors = "pt").to("cuda")
    
    _ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask, temperature=0.7,
                       streamer = text_streamer, max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id)
    print("\n" + "-"*100)
    print("Expected Response:")
    print(expected_response)

In [26]:
generate_response(ds["Prompt"][0], ds["Response"][0])

### **Routine**  
- **Bedtime at 2:30 AM and wake time at 10:00 AM indicate a delayed sleep schedule, which can disrupt circadian rhythms.**  
- **Midsleep point at 6:15 AM is significantly later than the typical range (between 2:00-3:00 AM) for this age group.**  

### **Sleep Quality**  
- **Light sleep is 51%, which is elevated (normal: 40-55%), suggesting disrupted sleep architecture.**  
- **Deep sleep is 25%, which is within the expected range (20-40%), but too early in the day.**  
- **REM sleep is 24%, which is within the normal range (20-25%), indicating normal cognitive function.**  
- **No awakenings, which is unusual for this age group.**  

### **Alertness**  
- **No reported naps, which is typical of this age group, but given the unusual sleep timing, daytime drowsiness may still occur.**  

### **Timing**  
- **A bedtime of 2:30 AM is significantly later than the typical range (10:00 PM - 2:00 AM) for this age group.**  
- **Wake time of 10:00 AM is also outside the norm

In [30]:
generate_response(ds["Prompt"][10], ds["Response"][10])

Here's an analysis of your sleep data, focusing on potential causes:

**1. Extrinsic Factors (Very Likely):**

*   **Relevance:** The consumption of 25mg of caffeine and one alcoholic drink could be significantly impacting your sleep architecture.
*   **Evidence:** Alcohol disrupts sleep, particularly in the later half of the night, while caffeine can interfere with sleep onset and maintenance.
*   **Explanation:** The amount of caffeine and the single alcoholic drink could be disrupting your sleep.

**2. Homeostatic Drive (Possible):**

*   **Relevance:** The high deep sleep percentage (67%) could be related to a strong homeostatic sleep drive, potentially resulting in a compensatory period of deep sleep.
*   **Evidence:** While deep sleep is important, the extremely high percentage is unusual and warrants further investigation.
*   **Explanation:** The unusually high deep sleep percentage could be related to a strong homeostatic drive, potentially resulting in a compensatory period o

In [34]:
generate_response(ds["Prompt"][20], ds["Response"][20])

Okay, let's look at what might be affecting your sleep, focusing on the most likely culprits first:

**1. Homeostatic Drive (Very Likely):**

*   **Your body is trying to compensate for prior sleep deprivation or other factors, leading to a high percentage of deep sleep and low light sleep. This means your body is prioritizing deep sleep, possibly at the expense of other stages.**

*   **The elevated sleep efficiency (95%) suggests you are not spending excessive time awake in bed, which is a good sign.**

**2. Extrinsic Factors (Possible):**

*   **Caffeine and alcohol consumption may be subtly altering your sleep architecture.** If you consume caffeine, it may suppress lighter sleep stages and lead to sleep fragmentation. Similarly, even small amounts of alcohol can disrupt sleep later in the night, potentially affecting the balance of sleep stages.

**3. Circadian Rhythm (Possible):**

*   **Your regular sleep schedule is well-structured, but the later bedtime may be influencing your

In [37]:
generate_response(ds["Prompt"][27], ds["Response"][27])

Given the trainee's overweight BMI and slightly elevated blood pressure, a gradual and progressive approach to exercise is crucial to minimize cardiovascular strain and musculoskeletal injury risk.
<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Given the trainee's overweight BMI, slightly elevated blood pressure, and moderate stress levels, a gradual and progressive approach to exercise is crucial to minimize the risk of musculoskeletal injuries and cardiovascular strain.



In [38]:
generate_response(ds["Prompt"][35], ds["Response"][35])

Answer: (B) Abrupt cessation of heavy and prolonged consumption of alcohol
Explanation: Delirium tremens (DT) is a serious alcohol withdrawal syndrome that can occur in people who have been drinking heavily for a long time. The symptoms usually begin 7 to 10 days after the last drink and can last up to 4 weeks. Symptoms include confusion, hallucinations, tremors, and seizures. Other symptoms include nausea, vomiting, fever, and abdominal pain. The syndrome can be fatal if seizures occur. Treatment includes chlordiazepoxide (Librium) or diazepam (Valium), which are benzodiazepines. Ref: Harrison's Principles of Internal Medicine, 16th Edition, Page 1926; Harrison's Principles of Internal Medicine, 17th Edition, Page 1965; Harrison's Principles of Internal Medicine, 18th Edition, Page 1906<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (B) Abrupt cessation of heavy and prolonged consumption of alc

In [39]:
generate_response(ds["Prompt"][135], ds["Response"][135])

### Option:
(B) Hypothalamus<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (B) Hypothalamus


In [41]:
generate_response(ds["Prompt"][138], ds["Response"][138])

### Answer: Answer: (C) Chromosome replication without cell division<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (C) Chromosome replication without cell division


In [44]:
generate_response(ds["Prompt"][142], ds["Response"][142])

Answer: (B) the adhesive forces between water and the walls of the graduated cylinder are greater than the adhesive forces between the mercury and the walls of the graduated cylinder.<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (D) the mercury has strong cohesive than adhesive forces, whereas water has strong adhesive than cohesive ones.


In [47]:
generate_response(ds["Prompt"][147], ds["Response"][147])

### Answer:
### Option A
Explanation: Cell lineage tracing is a method of identifying the specific cell that has given rise to a particular cell line. This can be done by isolating cells from tissues and using techniques such as cell culture and molecular biology.<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (A) the cells giving rise to and derived from a specific cell are known


In [48]:
generate_response(ds["Prompt"][150], ds["Response"][150])

Answer: (D) Pulmonary vein<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (B) Coronary sinus


In [54]:
generate_response(ds["Prompt"][407], ds["Response"][407])

### Option:
(A) increased plasma glucose.<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (D) increased sympathetic nerve activity.


In [55]:
generate_response(ds["Prompt"][411], ds["Response"][411])

I can understand your concern. Yes, you should definitely consult your doctor and discuss about changing the dose of lisinopril. Lisinopril is known to cause fluid retention and high blood pressure. So it is better to take low dose of lisinopril and add another antihypertensive drug like amlodipine or telmisartan. You may need diuretics like furosemide or spironolactone. So discuss all these with your doctor and start appropriate antihypertensive drugs. Don't worry, you will be alright. Hope I have solved your query. I will be happy to help you further. Wish you good health.<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
hello,You spaced out medicine, like take lisinopril in D morning and atenolol in the evening. Try to avoid to stress.   You should have a healthy lifestyle like avoiding fatty, oily and high calorie diet. Have low salt diet and monitor blood pressure regularly. Regular exercises like br

In [56]:
generate_response(ds["Prompt"][1175], ds["Response"][1175])

Answer: (D) Toxic epidermal necrolysis<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (D) Toxic epidermal necrolysis


In [59]:
generate_response(ds["Prompt"][1182], ds["Response"][1182])

Answer: (A) yes
Explanation: In the present case, the acute fibrinous and organizing pneumonia was a manifestation of immune dysregulation. In our experience, the diagnosis of VSAA is difficult to establish in a child with a history of infectious disease. The absence of an etiologic agent, in combination with the histologic findings, is highly suggestive of VSAA.<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (A) yes
Explanation: Aplastic anemia is associated with a variety of autoimmune syndromes. The sequence of events in our patient suggests that the hepatic failure, AFOP, and the VSAA may all have been part of an autoimmune syndrome. AFOP could be the result of immune dysregulation in this pediatric case with favorable outcome after immunosuppressive therapy and hematopoietic stem cell transplantation.


## 3B Model

In [60]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "johnjehiel/personal-health-LLM-Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = HF_TOKEN
)

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Unsloth 2025.3.14 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [61]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
def generate_response(prompt_input, expected_response):
    print("Input Prompt: ")
    print(prompt_input)
    print("\n" + "-"*100)
    inputs = tokenizer(
    [
        prompt_template.format(
            [prompt_input],
            "", # input
            "", # output - leave this blank for generation
        )
    ], return_tensors = "pt").to("cuda")
    
    _ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask, temperature=0.7,
                       streamer = text_streamer, max_new_tokens = 512, pad_token_id = tokenizer.eos_token_id)
    print("\n" + "-"*100)
    print("Expected Response:")
    print(expected_response)

In [62]:
generate_response(ds["Prompt"][0], ds["Response"][0])

Input Prompt: 
You are a sleep medicine expert. You are given the following sleep data.
The user is Female, 33 years old.
Sleep Summary: 
Bedtime: 2021-09-30 02:30:00
Wakeup time: 2021-09-30 10:00:00
Sleep duration: 7.5
Sleep efficiency: 0.6
REM sleep percentage: 24
Deep sleep percentage: 25
Light sleep percentage: 51
Awakenings: 0.0
Caffeine consumption: 50.0
Alcohol consumption: 0.0
Smoking status: No
Exercise frequency: 1.0

List the most important insights. Identify all of the patterns of data that are likely out of the preferred range. Make sure to consider various sleep health dimensions: Routine, Sleep Quality, Alertness, Timing, Efficiency, and Duration. Add a heading for each dimension. Optionally (only do this if extremely important) add a heading called Other for anything else that doesn't fit the above categories. For Routine, consider the average bedtime, wake time, midsleep point and standard deviations of these, focus on the consistency of the routine, not timing. For Sl

In [63]:
generate_response(ds["Prompt"][10], ds["Response"][10])

Input Prompt: 
You are a sleep medicine expert. You are given the following sleep data. 
The user is Female, 41 years old. 
Sleep Summary:
Bedtime: 2021-12-17 23:00:00
Wakeup time: 2021-12-17 07:30:00
Sleep duration: 8.5
Sleep efficiency: 0.85
REM sleep percentage: 20
Deep sleep percentage: 67
Light sleep percentage: 13
Awakenings: 0.0
Caffeine consumption: 25.0
Alcohol consumption: 1.0
Smoking status: Yes
Exercise frequency: 0.0

Based on the data, we can get the following insights: 
# Sleep Insights Report

Here's an analysis of your sleep data:

**Routine:** Your bedtime is 11:00 PM and wake-up time is 7:30 AM. This single night's data suggests a consistent sleep schedule. More data points are needed to confirm long-term consistency and calculate standard deviations.

**Sleep Quality:** Your deep sleep percentage is very high at 67%. While deep sleep is important, this level is significantly above the typical range (13-23%) and warrants further investigation. Your REM sleep percenta

In [64]:
generate_response(ds["Prompt"][20], ds["Response"][20])

Input Prompt: 
You are a sleep medicine expert. You are given the following sleep data. 
The user is Male, 52 years old. 
Sleep Summary: 
Bedtime: 2021-02-07 00:00:00
Wakeup time: 2021-02-07 08:00:00
Sleep duration: 8.0
Sleep efficiency: 0.95
REM sleep percentage: 25
Deep sleep percentage: 55
Light sleep percentage: 20
Awakenings: 0.0
Caffeine consumption: 50.0
Alcohol consumption: 1.0
Smoking status: No
Exercise frequency: 3.0

Based on the data, we can get the following insights: 
## Routine  
- **Bedtime (12:00 AM) and wake-up time (8:00 AM) indicate a consistent sleep schedule. A regular routine helps regulate circadian rhythms and improves sleep quality.**  

## Sleep Quality  
- **Deep sleep (55%) is significantly above the typical range (15â€“25%). While deep sleep is beneficial, an excessive proportion may indicate sleep stage imbalance, often linked to prior sleep deprivation or certain medications.**  
- **Light sleep (20%) is below the normal range (45â€“55%). Light sleep is

In [65]:
generate_response(ds["Prompt"][27], ds["Response"][27])

Input Prompt: 
You are a NSCA and ACSM board-certified fitness trainer who specializes in athlete training performance and recovery. 
Age: 43  
Gender: Male
Sleep Duration: 6.5
Quality of Sleep: 6
Physical Activity Level: 45
Stress Level: 7
BMI Category: Overweight 
Blood Pressure: 130/85
Heart Rate: 72
Daily Steps: 6000

Are there any special precautions that should be taken into account when recommending a fitness program to avoid injury? Comment if the trainee has exceptional demographics (e.g. very old, very high BMI, very low BMI) that require special considerations. Write a single sentence. Avoid mentioning diseases


----------------------------------------------------------------------------------------------------
Given the trainee's overweight BMI, slightly elevated blood pressure, moderate stress, and lower daily step count, a fitness program should prioritize gradual progression, low-impact activities, and stress management techniques to minimize injury risk.
<|eot_id|>

--

In [66]:
generate_response(ds["Prompt"][35], ds["Response"][35])

Input Prompt: 
You are a medical expert specialized in Medicine. Answer the following question by selecting the correct or best option.

### Question:
What is the cause of delirium tremens in alcoholics?

### Options:
(A) Fatty liver
(B) Abrupt cessation of heavy and prolonged consumption of alcohol
(C) Gradual withdrawal of alcohol
(D) Small doses of consumption


----------------------------------------------------------------------------------------------------
Answer: (B) Abrupt cessation of heavy and prolonged consumption of alcohol
Explanation: Delirium tremens is a severe form of alcohol withdrawal. It is caused by sudden discontinuation of heavy and prolonged alcohol consumption. The onset of delirium tremens is usually within 2-4 days after the last drink. The symptoms include confusion, disorientation, agitation, hallucinations and autonomic hyperactivity.<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected 

In [67]:
generate_response(ds["Prompt"][135], ds["Response"][135])

Input Prompt: 
You are a medical expert specialized in anatomy. Answer the following question by selecting the correct option.

### Question:
Which of the following controls body temperature, sleep, and appetite?

### Options:
(A) Adrenal glands
(B) Hypothalamus
(C) Pancreas
(D) Thalamus


----------------------------------------------------------------------------------------------------
Answer: (B) Hypothalamus<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (B) Hypothalamus


In [68]:
generate_response(ds["Prompt"][138], ds["Response"][138])

Input Prompt: 
You are a medical expert specialized in college biology. Answer the following question by selecting the correct option.

### Question:
Which of the following is the most direct cause of polyteny in somatic cells of certain organisms?

### Options:
(A) RNA transcription
(B) Supercoiling of chromatin
(C) Chromosome replication without cell division
(D) Chromosome recombination


----------------------------------------------------------------------------------------------------
Answer: (C) Chromosome replication without cell division\n<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (C) Chromosome replication without cell division


In [69]:
generate_response(ds["Prompt"][142], ds["Response"][142])

Input Prompt: 
You are a medical expert specialized in college medicine. Answer the following question by selecting the correct option.

### Question:
A student observes that mercury forms a convex meniscus in the graduated cylinder but that water forms a concave one. This behavior is best explained by the fact that:

### Options:
(A) the two liquids are being kept in graduated cylinders made of different materials.
(B) the adhesive forces between water and the walls of the graduated cylinder are greater than the adhesive forces between the mercury and the walls of the graduated cylinder.
(C) the cohesive forces between two mercury atoms are stronger than the cohesive forces between two water molecules.
(D) the mercury has strong cohesive than adhesive forces, whereas water has strong adhesive than cohesive ones.


----------------------------------------------------------------------------------------------------
Answer: (B) the adhesive forces between water and the walls of the gradu

In [70]:
generate_response(ds["Prompt"][147], ds["Response"][147])

Input Prompt: 
You are a medical expert specialized in medical genetics. Answer the following question by selecting the correct option.

### Question:
Tracing of a cell lineage during development means that

### Options:
(A) the cells giving rise to and derived from a specific cell are known
(B) the sequence of the enhancers for developmental genes is known
(C) the regulatory genes for the organism have been genetically mapped
(D) cell components in the membrane involved in signaling have been isolated


----------------------------------------------------------------------------------------------------
Answer: (A) the cells giving rise to and derived from a specific cell are known<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (A) the cells giving rise to and derived from a specific cell are known


In [71]:
generate_response(ds["Prompt"][150], ds["Response"][150])

Input Prompt: 
You are a medical expert . Answer the following question by selecting the correct option.

### Question:
A healthy 23-year-old male is undergoing an exercise stress test as part of his physiology class. If blood were to be sampled at different locations before and after the stress test, which area of the body would contain the lowest oxygen content at both time points?

### Options:
(A) Inferior vena cava
(B) Coronary sinus
(C) Pulmonary artery
(D) Pulmonary vein


----------------------------------------------------------------------------------------------------
Answer: (A) Inferior vena cava<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (B) Coronary sinus


In [72]:
generate_response(ds["Prompt"][407], ds["Response"][407])

Input Prompt: 
You are a medical expert specialized in clinical knowledge. Answer the following question by selecting the correct option.

### Question:
During exercise, adrenaline secretion from the adrenal glands is stimulated by:

### Options:
(A) increased plasma glucose.
(B) increased plasma fatty acids.
(C) increased plasma ACTH.
(D) increased sympathetic nerve activity.


----------------------------------------------------------------------------------------------------
Answer: (D) increased sympathetic nerve activity.<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (D) increased sympathetic nerve activity.


In [73]:
generate_response(ds["Prompt"][411], ds["Response"][411])

Input Prompt: 
You are a compassionate and expert medical advisor specializing in personalized health assistance.
Analyze the context and the patient’s query carefully, considering their background and concerns.

### Patient Context:
What causes high blood pressure while on Lisinopril and Atenolol?

### Patient Query:
I have a long history of treated hypertension but recently my blood pressures have been high in the morning (150/100) but better as the day goes on (122/80). I take Lisinopril 20 mg and Atenolol 25 mg in the morning. Should I worry about this? Does it make any sense to divide the dose and take some medicine at night?

Provide a detailed, evidence-based, and empathetic response that offers practical recommendations.


----------------------------------------------------------------------------------------------------
I have reviewed your query and here is my advice. You should not change the dose of medications without doctor advise. It is better to take the medicines as p

In [74]:
generate_response(ds["Prompt"][1175], ds["Response"][1175])

Input Prompt: 
You are a medical expert specialized in professional medicine. Answer the following question by selecting the correct option.

### Question:
A 60-year-old man has had painful skin with exfoliation of the skin and mucous membranes for 1 day. He has been taking allopurinol and probenecid for 2 weeks because of gouty arthritis. There is diffuse exfoliation of the skin with oozing of serous fluid. The mucous membranes of the mouth are erythematous and exfoliated. There are no target lesions. Which of the following is the most likely diagnosis?

### Options:
(A) Erythema multiforme
(B) Pemphigus erythematosus
(C) Staphylococcal scalded-skin syndrome
(D) Toxic epidermal necrolysis


----------------------------------------------------------------------------------------------------
Answer: (D) Toxic epidermal necrolysis<|eot_id|>

----------------------------------------------------------------------------------------------------
Expected Response:
Answer: (D) Toxic epidermal 

In [75]:
generate_response(ds["Prompt"][1182], ds["Response"][1182])

Input Prompt: 
You are a medical expert specialized in bio-medical research. Analyze the given context and answer the following question by selecting the correct or best option.

### Context:
Acute fibrinous and organizing pneumonia (AFOP) is a recently described histologic pattern of diffuse pulmonary disease. In children, all cases reported to date have been fatal. In this study, we describe the first nonfatal AFOP in a child and review the literature.
A 10-year-old boy developed very severe aplastic anemia (VSAA) after being admitted to our hospital with a fulminant hepatic failure of unknown origin. A chest computed tomography scan revealed multiple lung nodules and a biopsy of a pulmonary lesion showed all the signs of AFOP. Infectious workup remained negative. We started immunosuppressive therapy with antithymocyte globulin and cyclosporine to treat VSAA. Subsequent chest computed tomography scans showed a considerable diminution of the lung lesions but the VSAA did not improve u